## <span style="color:Orange"> **Import Packages and Settings** </span>

In [ ]:
import json
import pandas as pd
import os
import re
import numpy as np
from sklearn import naive_bayes
pd.set_option('display.max_rows', 100)

from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk import bigrams

import sklearn.model_selection as ms
import random

import matplotlib.pyplot as plt
import matplotlib as matplotlib
import os

from pylab import figure, axes, pie, title, show

import pickle
import sys as sys

## <span style="color:Orange"> **Import Data** </span>

In [ ]:
data = pd.read_json('./train_data/merchantSwipeDump_old.json')
data.head()

## <span style="color:Orange"> **Import Data Cleaning Module and Clean** </span>

In [ ]:
import sys
sys.path.insert(0, '../utility_files')
import cleanData

cleanData.clean_help()

remove_string = '[]' #'^[0-9]*[0-9]$|^www$|^com$|^ave|^street$|^st$|^road$|^and$|^inc$|^at$|^drive$|^of$|^main$|^the$|^[ewns]$|^#|^[0-9]*th$|^[0-9]*rd$|^1st$|^store$|^south$'
split_string = "[- ./*']"

cleanData.clean(data,rejoin_col_strings_bi=True,lowercase=True,split_string="[- ./*']",remove_string = '[]',join_mcc_bi=True) 

## <span style="color:Orange"> **Import Dictionary Labels to Expand Train** </span>

In [ ]:
data = data.reset_index()
data.columns = ['Index'] + list(data.columns[1:])

labels = pd.read_csv('./dictionary/dictionary_labels.csv')
labels.index = labels.index.set_names(['Index'])
labels.reset_index()
labels.columns = ['Index'] + list(labels.columns[1:])
labels = labels[['Index','mapped_brand_dict_3']]
data  = pd.merge(data, labels, how='left', on= 'Index')
data = data.fillna('None')
data.sample(10)

## <span style="color:Orange"> **Change mapped_brand to the dictionary guess when None** </span>

In [ ]:
data['mapped_brand_guess'] = np.where(data['mapped_brand']=='None',data['mapped_brand_dict_3'],data['mapped_brand'])
# data['mapped_brand_dict_3'] if data['mapped_brand']=='None' else data['mapped_brand']
print(len(data[data.mapped_brand_guess!='None']) -len(data[data.mapped_brand_dict_3!='None']))
print(len(data[data.mapped_brand_dict_3!='None']))
print(len(data[data.mapped_brand!='None']))
data.fillna('None_',inplace=True)

## <span style="color:Orange"> **Import Train Test Split Module** </span>

In [ ]:
sys.path.insert(0, '../utility_files')
import stratifiedSplits

stratifiedSplits.stratfiedTrainTestSplit_help()

hasLabel,noLabel,X_train, X_test, y_train, y_test = stratifiedSplits.stratfiedTrainTestSplit(df=data, brandCol ='mapped_brand_guess', holdoutPct = .25, ones = 'train')

#noLabel = data[data['mapped_brand'].isnull()]
noLabel = data[data['mapped_brand_guess']=='None']


## <span style="color:Orange"> **Import NB Module and Run it** </span>

In [ ]:
import time
start_time = time.time()

import pickle

sys.path.insert(0, '../utility_files')
import NaiveBayes


NaiveBayes.NB(writeResults = False,makeAllPredictions = True,cleanMerchantCol = 'merchant_string_clean',probThresholdAnalysis=True,
              hasLabel=hasLabel,noLabel=noLabel,X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

print("--- %s seconds ---" % (time.time() - start_time))